# Three dimensional visualisation

OptimUS provides simulations of harmonic acoustic waves. This tutorial shows visualise functionality in three dimensions. We use an field focused inside a ribcage model. 

## Import the OptimUS library

Load the OptimUS library.

In [1]:
import sys
sys.path.append('..')

In [2]:
import optimus

## Define the physical parameters and the geometry of the problem

We consider a beam from a bowl source at 100 kHz, focused at a distance of 64 mm. Furthermore, the exterior material is water and the interior material is cortical bone.

In [3]:
frequency = 100e3
radius_of_curvature = 64e-3
radius = radius_of_curvature / 2
location = (0, 0, -radius_of_curvature)
velocity = 0.04
source = optimus.source.create_bowl(frequency, outer_radius=radius, radius_of_curvature=radius_of_curvature, location=location, source_axis=(0, 0, 1), velocity=velocity)

In [4]:
material_ext = optimus.material.load_material("water")
material_int = optimus.material.load_material("bone (cortical)")

We use a model with four ribs for the focused ultrasound simulation. Let us consider the full model at low mesh density and a shortened version with only three ribs with a finer mesh.

In [5]:
geometry1 = optimus.geometry.load.import_grid("Data/ribcage4-h-4.msh", label="ribcage4")
geometry2 = optimus.geometry.load.import_grid("Data/ribcage3-h-1.msh", label="ribcage3")

We like to verify the meshes before solving the BEM model. For this purpose, let us use the object `Visualise3DGrid` in OptimUS. It takes a list of the geometry objects to display. The control panel displayed on the chart allows to change the grid color, opacity, wireframe and other settings. For example, we notice that the units of the ribcage are in millimeters. Furthermore, we see that the finer mesh is indeed part of the larger ribcage.

In [6]:
grids = optimus.postprocess.Visualise3DGrid([geometry1, geometry2])
grids.create_computational_grid()
grids.display_field()

/usr/local/lib/python3.5/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/usr/local/lib/python3.5/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

## Solve the acoustic model

As noticed in the visualisation, the mesh file comes in millimeters, while OptimUS requires standard units. Hence, let us scale the mesh to meters and create a default acoustic model.

In [7]:
geometry = optimus.utils.mesh.scale_mesh(geometry1, 1e-3)
model = optimus.model.create_default_model(source, geometry, material_ext, material_int)

Be careful that solving the model may take several minutes, depending on the computer facility.

In [8]:
%%time
model.solve()

CPU times: user 13min 47s, sys: 3min 52s, total: 17min 40s
Wall time: 33.8 s


After solving the scattering model, let us compute the field in a plane through the focal point.

In [9]:
optimus.global_parameters.postprocessing.quadrature_order = 20
postprocess_plane = optimus.postprocess.VisualisePlane(model)
postprocess_plane.create_computational_grid(resolution=(141, 241), bounding_box=(-90e-3, 90e-3, -90e-3, 90e-3),
                                            plane_axes=(1, 2))
postprocess_plane.compute_fields()

## Visualise the field in 3D

We display the surface and plane with their respective field values using the object `Visualise3DField`. It is possible to change the grid color, opacity, wireframe and other settings with the control panel displayed on the chart.

In [10]:
postprocess_3D = optimus.postprocess.Visualise3DField(model)
postprocess_3D.create_computational_grid()
postprocess_3D.add_VisualisePlane(postprocess_plane)
postprocess_3D.compute_fields()
postprocess_3D.display_field()

Output()

## Visualisation in a cloud of points

The acoustic field can be calculated on any list of points in space. With the following function it is possible to display a three-dimensional visualisation for any chosen set of points on the space. For illustration purposes, we use points randomnly located in a space near the ribcage.

In [11]:
import numpy as np

In [12]:
n = 500
pointsarray = np.random.rand(3,n) * 0.18 - 0.09

We plot the cloud of points with colors corresponding to the magnitude of the field in that position using the function `VisualiseCloudPoints`. It is possible to change the grid color, opacity, wireframe and other settings with the control panel displayed on the chart.

In [13]:
points = optimus.postprocess.VisualiseCloudPoints(model)
points.create_computational_grid(pointsarray)
points.compute_fields()
points.display_field(size = 0.009)

Output()